<a href="https://colab.research.google.com/github/HendricoYehezky/Tugas-1-Deep-Learning/blob/main/Bab_5_1_Layers_dan_Blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l

     |████████████████████████████████| 82 kB 343 kB/s 
     |████████████████████████████████| 9.9 MB 30.2 MB/s 
     |████████████████████████████████| 20.1 MB 15.9 MB/s 
     |████████████████████████████████| 61 kB 9.6 MB/s 
     |████████████████████████████████| 11.6 MB 52.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the 

In [2]:
### Layers dan Blocks ###
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1341, -0.0336, -0.3205, -0.0282,  0.0058, -0.1366,  0.0224,  0.0665,
          0.1761, -0.2070],
        [-0.1475,  0.1260, -0.2724, -0.1058, -0.0783, -0.1172, -0.1482,  0.0951,
          0.1519, -0.2241]], grad_fn=<AddmmBackward0>)

In [3]:
### Custom Blok ###
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.0186, -0.1822,  0.0977, -0.0657,  0.0445, -0.0790, -0.0845, -0.0109,
          0.0778, -0.0927],
        [-0.1221, -0.1365,  0.2151, -0.0698,  0.0320, -0.1040,  0.0379,  0.0860,
          0.1274, -0.1282]], grad_fn=<AddmmBackward0>)

In [5]:
### Blok Sekuensial ###
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1002,  0.0344,  0.0358, -0.0497,  0.1846, -0.0816,  0.3047, -0.2022,
          0.0030,  0.1839],
        [-0.0074, -0.0692,  0.0315, -0.1153,  0.1787, -0.0643,  0.2914, -0.2302,
          0.0842,  0.1975]], grad_fn=<AddmmBackward0>)

In [7]:
### Mengeksekusi Kode di dalam Fungsi Propagation Forward ###
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1115, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1010, grad_fn=<SumBackward0>)